In [1]:
import pandas as pd
from tqdm import tqdm

train = pd.read_csv('../input/feedback-prize-2021/train.csv')
train.head()

,id,discourse_id,discourse_start,discourse_end,discourse_text,discourse_type,discourse_type_num,predictionstring
0,423A1CA112E2,1.622628e+12,8.0,229.0,Modern humans today are always on their phone....,Lead,Lead 1,1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 1...
1,423A1CA112E2,1.622628e+12,230.0,312.0,They are some really bad consequences when stu...,Position,Position 1,45 46 47 48 49 50 51 52 53 54 55 56 57 58 59
2,423A1CA112E2,1.622628e+12,313.0,401.0,Some certain areas in the United States ban ph...,Evidence,Evidence 1,60 61 62 63 64 65 66 67 68 69 70 71 72 73 74 75
3,423A1CA112E2,1.622628e+12,402.0,758.0,"When people have phones, they know about certa...",Evidence,Evidence 2,76 77 78 79 80 81 82 83 84 85 86 87 88 89 90 9...
4,423A1CA112E2,1.622628e+12,759.0,886.0,Driving is one of the way how to get around. P...,Claim,Claim 1,139 140 141 142 143 144 145 146 147 148 149 15...


In [2]:
classes = train.discourse_type.unique().tolist()
classes

['Lead',
 'Position',
 'Evidence',
 'Claim',
 'Concluding Statement',
 'Counterclaim',
 'Rebuttal']

In [3]:
from collections import defaultdict
tags = defaultdict()

for i, c in enumerate(classes):
    tags[f'B-{c}'] = i
    tags[f'I-{c}'] = i + len(classes)
tags[f'O'] = len(classes) * 2
tags[f'Special'] = -100
    
l2i = dict(tags)

i2l = defaultdict()
for k, v in l2i.items(): 
    i2l[v] = k
i2l[-100] = 'Special'

i2l = dict(i2l)

N_LABELS = len(i2l) - 1 # not accounting for -100

In [4]:
l2i

{'B-Lead': 0,
 'I-Lead': 7,
 'B-Position': 1,
 'I-Position': 8,
 'B-Evidence': 2,
 'I-Evidence': 9,
 'B-Claim': 3,
 'I-Claim': 10,
 'B-Concluding Statement': 4,
 'I-Concluding Statement': 11,
 'B-Counterclaim': 5,
 'I-Counterclaim': 12,
 'B-Rebuttal': 6,
 'I-Rebuttal': 13,
 'O': 14,
 'Special': -100}

In [5]:
test_names, train_texts = [], []
for f in tqdm(list(os.listdir('../input/feedback-prize-2021/train'))):
    test_names.append(f.replace('.txt', ''))
    train_texts.append(open('../input/feedback-prize-2021/train/' + f, 'r', encoding='utf-8').read())
train_text_df = pd.DataFrame({'id': test_names, 'text': train_texts})
train_text_df.head()

100%|██████████| 15594/15594 [00:00<00:00, 36335.62it/s]


,id,text
0,4BCA0580352F,Isn't it wonderful to know that you get a chan...
1,29A8E6DA6F04,"One uses a car to go to the store, pick someon..."
2,17E8BB6C550B,Driverless cars do seem like the thing today's...
3,319B2511943D,Have you ever thought your choice wasn't the b...
4,C9B1827498E4,"Imagine you're sitting at home, watching a sho..."


In [6]:
all_entities = []
all_text_list = []
for ii,i in enumerate(train_text_df.iterrows()):
    if ii%100==0: print(ii,', ',end='')
    total = i[1]['text'].split().__len__()
    text_list = i[1]['text'].split()
    entities = ["O"]*total
    for j in train[train['id'] == i[1]['id']].iterrows():
        discourse = j[1]['discourse_type']
        list_ix = [int(x) for x in j[1]['predictionstring'].split(' ')]
        entities[list_ix[0]] = f"B-{discourse}"
        for k in list_ix[1:]: entities[k] = f"I-{discourse}"
    all_entities.append(entities)
    all_text_list.append(text_list)
train_text_df['entities'] = all_entities
train_text_df['text_list'] = all_text_list

0 , 100 , 200 , 300 , 400 , 500 , 600 , 700 , 800 , 900 , 1000 , 1100 , 1200 , 1300 , 1400 , 1500 , 1600 , 1700 , 1800 , 1900 , 2000 , 2100 , 2200 , 2300 , 2400 , 2500 , 2600 , 2700 , 2800 , 2900 , 3000 , 3100 , 3200 , 3300 , 3400 , 3500 , 3600 , 3700 , 3800 , 3900 , 4000 , 4100 , 4200 , 4300 , 4400 , 4500 , 4600 , 4700 , 4800 , 4900 , 5000 , 5100 , 5200 , 5300 , 5400 , 5500 , 5600 , 5700 , 5800 , 5900 , 6000 , 6100 , 6200 , 6300 , 6400 , 6500 , 6600 , 6700 , 6800 , 6900 , 7000 , 7100 , 7200 , 7300 , 7400 , 7500 , 7600 , 7700 , 7800 , 7900 , 8000 , 8100 , 8200 , 8300 , 8400 , 8500 , 8600 , 8700 , 8800 , 8900 , 9000 , 9100 , 9200 , 9300 , 9400 , 9500 , 9600 , 9700 , 9800 , 9900 , 10000 , 10100 , 10200 , 10300 , 10400 , 10500 , 10600 , 10700 , 10800 , 10900 , 11000 , 11100 , 11200 , 11300 , 11400 , 11500 , 11600 , 11700 , 11800 , 11900 , 12000 , 12100 , 12200 , 12300 , 12400 , 12500 , 12600 , 12700 , 12800 , 12900 , 13000 , 13100 , 13200 , 13300 , 13400 , 13500 , 13600 , 13700 , 13800 , 

In [7]:
print( train_text_df.shape)
train_text_df.head()

(15594, 4)


,id,text,entities,text_list
0,4BCA0580352F,Isn't it wonderful to know that you get a chan...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea...","[Isn't, it, wonderful, to, know, that, you, ge..."
1,29A8E6DA6F04,"One uses a car to go to the store, pick someon...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea...","[One, uses, a, car, to, go, to, the, store,, p..."
2,17E8BB6C550B,Driverless cars do seem like the thing today's...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea...","[Driverless, cars, do, seem, like, the, thing,..."
3,319B2511943D,Have you ever thought your choice wasn't the b...,"[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea...","[Have, you, ever, thought, your, choice, wasn'..."
4,C9B1827498E4,"Imagine you're sitting at home, watching a sho...","[B-Lead, I-Lead, I-Lead, I-Lead, I-Lead, I-Lea...","[Imagine, you're, sitting, at, home,, watching..."


In [8]:
from datasets import Dataset

ds = Dataset.from_pandas(train_text_df)
datasets = ds.train_test_split(test_size = 0.1, shuffle=True, seed=42)
datasets

DatasetDict({
    train: Dataset({
        features: ['id', 'text', 'entities', 'text_list'],
        num_rows: 14034
    })
    test: Dataset({
        features: ['id', 'text', 'entities', 'text_list'],
        num_rows: 1560
    })
})

In [9]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained('allenai/longformer-base-4096', add_prefix_space=True)

Downloading:   0%|          | 0.00/694 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/878k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/446k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.29M [00:00<?, ?B/s]

In [10]:
def fix_beginnings(labels):
    for i in range(1,len(labels)):
        curr_lab = labels[i]
        prev_lab = labels[i-1]
        if curr_lab in range(7,14):
            if prev_lab != curr_lab and prev_lab != curr_lab - 7:
                labels[i] = curr_lab -7
    return labels

def preparing_train_dataset(examples):
    encoding = tokenizer(examples['text_list'], truncation=True, padding=True, max_length = 1024, is_split_into_words=True)
    total= len(encoding['input_ids'])
    encoding['labels']=[]
    for i in range(total):
        labels = [l2i['O'] for _ in range(len(encoding['input_ids'][i]))]
        word_idx = encoding.word_ids(batch_index=i)
        for j in range(len(word_idx)):
            if word_idx[j] is None:
                labels[j]=l2i['Special']
            else:
                label = examples['entities'][i][word_idx[j]]
                if label[0]=='B' and j!=0 and word_idx[j-1]==word_idx[j]:
                    label_t = label.replace('B', 'I')
                    labels[j]=l2i[label_t]
                else:
                    labels[j]=l2i[label]
        labels = fix_beginnings(labels)
        encoding['labels'].append(labels)
    return encoding

In [11]:
tokenized_datasets = datasets.map(preparing_train_dataset, batched=True, batch_size=10000, remove_columns=datasets["train"].column_names)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [12]:
from transformers import AutoModelForTokenClassification, TrainingArguments, Trainer, AutoConfig

config = AutoConfig.from_pretrained('allenai/longformer-base-4096')
config.num_labels = N_LABELS
model = AutoModelForTokenClassification.from_pretrained('allenai/longformer-base-4096', config = config)

Downloading:   0%|          | 0.00/570M [00:00<?, ?B/s]

Some weights of the model checkpoint at allenai/longformer-base-4096 were not used when initializing LongformerForTokenClassification: ['lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.dense.weight']
- This IS expected if you are initializing LongformerForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing LongformerForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of LongformerForTokenClassification were not initialized from the model checkpoint at allenai/longformer-base-4096 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN

In [25]:
training_args = TrainingArguments(
    output_dir = './output/longformer-baseline',
    evaluation_strategy = 'steps',
    per_device_train_batch_size = 8,
    per_device_eval_batch_size = 8,
    gradient_accumulation_steps = 2,
    learning_rate = 1e-5,
    weight_decay = 0.01,
    max_grad_norm = 10,
    num_train_epochs = 5,
    warmup_ratio = 0.1,
    logging_strategy = 'steps',
    logging_steps = 10,
    save_strategy = 'steps',
    save_total_limit = 1,
    seed = 42,
    eval_steps = 50,
    save_steps = 50,
    dataloader_num_workers = 2,
    load_best_model_at_end = True,
    metric_for_best_model = 'f1',# need to fix
    group_by_length = True,
    report_to = 'wandb',
)

PyTorch: setting up devices


In [26]:
from transformers import DataCollatorForTokenClassification

data_collator = DataCollatorForTokenClassification(tokenizer)

In [27]:
import numpy as np
from datasets import load_metric

metric = load_metric('seqeval')

def compute_metrics(p):
    predictions, labels = p
    predictions = np.argmax(predictions, axis=2)
    
    true_predictions = [
        [i2l[p] for (p, l) in zip(prediction, label) if l!= -100] for prediction, label in zip(predictions, labels)
    ]
    true_labels = [
        [i2l[l] for (p, l) in zip(prediction, label) if l!= -100] for prediction, label in zip(predictions, labels)
    ]
    
    results = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": results["overall_precision"],
        "recall": results["overall_recall"],
        "f1": results["overall_f1"],
        "accuracy": results["overall_accuracy"],
    }

In [28]:
trainer = Trainer(
    model,
    training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    data_collator=data_collator,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics, 
)

In [29]:
import wandb

run = wandb.init(project='Feedback-prize', entity='donggunseo', name='longformer-baseline')
os.environ["TOKENIZERS_PARALLELISM"] = "false"
# os.environ["CUDA_VISIBLE_DEVICES"]="0,2"
trainer.train()
run.finish()
trainer.save_model('longformer-baseline_best')

***** Running training *****
  Num examples = 14034
  Num Epochs = 5
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 48
  Gradient Accumulation steps = 2
  Total optimization steps = 1460
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
50,1.704700,1.612643,0.002784,0.000346,0.000616,0.540288
100,1.530000,1.431925,0.001267,0.000346,0.000544,0.540638
150,1.077600,0.971407,0.009354,0.015795,0.011750,0.706571
200,0.943600,0.868745,0.009739,0.020159,0.013133,0.731665
250,0.906300,0.793720,0.026070,0.047246,0.033600,0.749078
300,0.862700,0.760780,0.033579,0.071216,0.045639,0.758025
350,0.787000,0.730735,0.042784,0.078836,0.055466,0.765646
400,0.752200,0.716392,0.057560,0.110980,0.075804,0.769912
450,0.740500,0.688651,0.072217,0.141046,0.095524,0.775784
500,0.743500,0.678463,0.078607,0.161967,0.105845,0.777663


***** Running Evaluation *****
  Num examples = 1560
  Batch size = 24
/usr/local/lib/python3.6/dist-packages/seqeval/metrics/v1.py:57: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
Saving model checkpoint to ./output/longformer-baseline/checkpoint-50
Configuration saved in ./output/longformer-baseline/checkpoint-50/config.json
Model weights saved in ./output/longformer-baseline/checkpoint-50/pytorch_model.bin
tokenizer config file saved in ./output/longformer-baseline/checkpoint-50/tokenizer_config.json
Special tokens file saved in ./output/longformer-baseline/checkpoint-50/special_tokens_map.json
/usr/local/lib/python3.6/dist-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  

eval/accuracy,▁▁▆▆▇▇▇▇█████████████████████
eval/f1,▁▁▁▁▂▃▃▄▅▅▅▆▇▇▇▇▇▇▇█▇▇███████
eval/loss,█▇▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
eval/precision,▁▁▁▁▂▃▃▄▅▅▅▆▇▇▇▆▇▇▇█▇▇▇██████
eval/recall,▁▁▁▁▂▃▃▄▅▅▅▆▇▇▆▇▇▇▇▇█▇███████
eval/runtime,▁▁▂▄▅▃█▆▁▄▅▄▅▃▅▃▂▂▅▆▄▃▄▂▄▁▄▄▄
eval/samples_per_second,██▇▅▄▆▁▃█▅▄▅▄▆▄▆▇▇▄▃▅▆▅▆▅█▅▅▅
eval/steps_per_second,██▇▅▄▆▁▃█▅▄▅▄▆▄▆▇▇▄▃▅▆▅▆▅█▅▅▅
train/epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/global_step,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,▂▃▅▇███▇▇▇▇▇▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▁▁▁


Saving model checkpoint to longformer-baseline_best
Configuration saved in longformer-baseline_best/config.json
Model weights saved in longformer-baseline_best/pytorch_model.bin
tokenizer config file saved in longformer-baseline_best/tokenizer_config.json
Special tokens file saved in longformer-baseline_best/special_tokens_map.json


In [22]:
torch.cuda.empty_cache()